In [238]:
import scipy.io
import glob
import torch 
import numpy as np
import torch 
import re
import shutil
import os

In [239]:
skeleton_file_path = glob.glob('Datasets/UTD_MHAD_random_split/test_skeleton/*.mat')
inertia_file_path = glob.glob('Datasets/UTD_MHAD_random_split/test_inertial/*.mat')

In [241]:
len(skeleton_file_path)

109

In [242]:
file_map = {}
for i in range(len(inertia_file_path)):
    identity = inertia_file_path[i].split('/')[-1].split('_')[:-1]
    identity = ('').join(identity)
    file_map[identity] = i


In [243]:
def get_unique_id(file_path):
    file_name = file_path.split('/')[-1]
    splited_array = file_name.split('_')
    u_id = ('').join(splited_array[:-1])
    act_id = splited_array[0]
    return u_id,act_id

In [244]:
frames = 100
num_joints = 20
acc_frames = 150
channels = 3
dataset = np.zeros((len(skeleton_file_path), frames , num_joints + acc_frames, channels))
labels = np.zeros(len(skeleton_file_path))
for ind in range(len(skeleton_file_path)):

    skeleton_data = scipy.io.loadmat(skeleton_file_path[ind])['d_skel']

    #calculate the max and min along each direction 
    

    u_id, act_id = get_unique_id(skeleton_file_path[ind])
    row, column, chanel = skeleton_data.shape
    new_x = np.linspace(0, chanel, frames)
    interpolated_skel = np.zeros((frames,row, column))
    x_coords = np.arange(chanel)
    inertial_file_index = file_map[u_id]
    
#     try:
#         inertial_file_index = file_map[u_id]
#         print(skeleton_file_path[ind])
        
#     except: 
#         os.remove(skeleton_file_path[ind])
#         continue
        
    for i in range(row):
        for j in range(column):
            data = skeleton_data[i, j, :]
            new_data = np.interp(new_x,x_coords,data)
            interpolated_skel[:,i, j] = new_data
            

    

    inertia_data = scipy.io.loadmat(inertia_file_path[inertial_file_index])['d_iner']
    inr_r, inr_c = inertia_data[:, :3].shape

    new_inr_x = np.linspace(0, inr_r, acc_frames)
    old_x = np.arange(inr_r)
    interpolated_inr = np.zeros((acc_frames, inr_c))
    
    #interpolating every column
    for j in range(inr_c):
            data = inertia_data[:, j]
            interpolated_inr[:, j] = np.interp(new_inr_x, old_x, data)
    
    acc_ext = np.zeros((frames, acc_frames, column))
    acc_ext[0, : , :] = interpolated_inr
    
    merged = np.concatenate((interpolated_skel,acc_ext,), axis= 1)
    dataset[ind,:,:,:] = merged
    match = re.search(r'\d+', act_id)
    labels[ind] = int(match.group())
    

In [245]:
np.savez('randtest_data.npz', data=dataset, labels=labels)

In [196]:
dataset.shape

(643, 100, 170, 3)

In [38]:
skeleton_file_path = glob.glob('Datasets/UTD_MAAD/Inertial/*s3*.mat')

In [39]:
destination_folder = 'Datasets/UTD_MAAD/test_inertial/'

In [189]:
training_set = Utd_Dataset('Datasets/UTD_MAAD/train_data.npz')
training_generator = torch.utils.data.DataLoader(training_set, **params) #Each produced sample is  200 x 59 x 3

In [180]:

import random

In [181]:
inertial_paths = glob.glob('Datasets/UTD_MHAD_random_split/train_inertial/**')
skeletal_paths = glob.glob('Datasets/UTD_MHAD_random_split/train_skeleton/**')

In [182]:
rand_num = random.sample(range(len(inertial_paths)), 109)

In [183]:
if os.path.isdir('Datasets/UTD_MHAD_random_split/test_inertial') != True: 
    os.mkdir('Datasets/UTD_MHAD_random_split/test_inertial')
if os.path.isdir('Datasets/UTD_MHAD_random_split/test_skeleton') != True:
    os.mkdir('Datasets/UTD_MHAD_random_split/test_skeleton')
    
for i in rand_num:
    inertial_path = inertial_paths[i]
    name = '_'.join(inertial_path.split('/')[-1].split('_')[:-1])
    skeleton_path = 'Datasets/UTD_MHAD_random_split/train_skeleton/'+name+'_'+'skeleton.mat'
    shutil.move(inertial_paths[i], 'Datasets/UTD_MHAD_random_split/test_inertial')    
    shutil.move(skeleton_path, 'Datasets/UTD_MHAD_random_split/test_skeleton')
    

In [72]:
inertial_file_path = sorted(inertial_paths)
skeletal_paths = sorted(skeletal_paths)

In [83]:
a = inertial_file_path[1]

In [85]:
'_'.join(a.split('/')[-1].split('_')[:-1]) +'_'+'skeleton.mat'

'a10_s1_t2_skeleton.mat'

In [50]:
732*.15

109.8